<a href="https://colab.research.google.com/github/MohammadDeep/vision/blob/main/vision_dataset_model_classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from Config import image_dir, image_val_dir,ann_file, ann_file_val



In [ ]:
from pycocotools.coco import COCO
if __name__ == "__main__":
  # بارگذاری فایل آنوتیشن COCO (آدرس فایل خود را قرار دهید)
  coco = COCO(ann_file_val)  # آدرس فایل آنوتیشن خود را قرار دهید

  # استخراج لیست کلاس‌ها (دسته‌بندی‌ها)
  categories = coco.loadCats(coco.getCatIds())


  print(categories)


loading annotations into memory...
Done (t=18.63s)
creating index...
index created!
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'vehicle', 'id': 3, 'name': 'car'}, {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}, {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}, {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}, {'supercategory': 'vehicle', 'id': 7, 'name': 'train'}, {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}, {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}, {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}, {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}, {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}, {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'}, {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'}, {'supercategory': 'animal', 'id': 16, 'name': 'bird'}, {'supercategory': 'animal', 'id': 17, '

# create folber dataset

In [ ]:
from pycocotools.coco import COCO
import json
import os
import requests
import random
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import os
import requests
from pycocotools.coco import COCO
import shutil

In [ ]:


class getDataFoalberCoco:
  def __init__(self,
               image_dir: str ,
               ann_file_1 : str,
               list_catecoy : list,
               destination_dir : str
                ):
    self.image_dir = image_dir
    self.ann_file  = ann_file_1
    self.list_catecoy = list_catecoy
    self.destination_dir = destination_dir


  def get_image_label(self):
    '''
    get data and label in list
    self.list_data[{'image':img,'anns':anns}, ...]
    '''
    if  hasattr(self, 'list_data'):
      return None
    print('in function   :  get_image_lable')
    with open(self.ann_file, 'r') as f:
      data = json.load(f)
    coco = COCO(self.ann_file)
    self.list_data = []
    for i in range(len(data['images'])):
      # Find image ID based on file name
      image_id = data['images'][i]['id']
      '''
      for image_info in data['images']:
          if image_info['file_name'] == '000000397133.jpg':
              image_id = image_info['id']
              break
      '''
      if image_id is not None:
          # Load image data using the found image ID
          img = coco.loadImgs(image_id)[0]
          anns = coco.loadAnns(coco.getAnnIds(imgIds=img['id']))
          #cats = coco.loadCats(coco.getCatIds())
          #labels = [cat['name'] for cat in cats]

          self.list_data.append({'image':img,'anns':anns})
      else:
          print(f"Image with number image  {i}  not found in the dataset.")




  def get_category_name(self):
    '''
     list of category_id
    -> self.caregory_list
    '''
    if   hasattr(self, 'category_list') :
      return None
    print('in function   :  get_category_name')
    self.get_image_label()
    self.category_list = []

    for i  in self.list_data:
      list_new = [i1['category_id']for i1 in i['anns']]
      self.category_list.append(list_new)





  def copy_image(self, data, categore_name : str):
      """Copies images to a new directory based on the provided list_data.
      Args:
          list_data: The list of image data from the COCO dataset.
          destination_dir: The directory to copy the images to.
      """
      destination_dir1 = os.path.join(self.destination_dir, categore_name)
      if not os.path.exists(destination_dir1):
          os.makedirs(destination_dir1)



      image_info = data['image']
      image_path = os.path.join(self.image_dir, image_info['file_name'])

      if os.path.exists(image_path):
          destination_path = os.path.join(destination_dir1, image_info['file_name'])
          shutil.copy2(image_path, destination_path) # copy2 preserves metadata
          #print(f"Copied {image_info['file_name']} to {destination_path}")
      else:
          print(f"Warning: Image file not found: {image_path}")


  def get_new_category_list(self):
    '''
    -> self.new_category_list
    '''
    if hasattr(self, 'new_category_list'):
      return None
    print('in function   :  get_new_category_list')
    self.get_category_name()
    self.new_category_list = []
    for i in self.category_list:

      a = np.array(i)
      b = np.array(self.list_catecoy)

      label = np.intersect1d(a, b)
      label = label if label.size > 0 else [-1]
      self.new_category_list.append(label)


  def create_dataset_folber(self):
    '''
    create and save dataset image folber
    '''
    self.get_new_category_list()
    len_all_data  = len(self.new_category_list)
    for i in tqdm(range(len_all_data)):
      for i1 in self.new_category_list[i]:
        self.copy_image(data = self.list_data[i], categore_name  =  str(i1))





In [ ]:

if __name__ == "__main__":


  '''

  train_data = getDataFoalberCoco(
               image_dir  ,
               ann_file,
               [1],
               '/content/data_set/train_dataset')

  train_data.create_dataset_folber()

  '''
  val_data =  getDataFoalberCoco(
               image_val_dir  ,
               ann_file_val,
               [1],
               '/content/data_set/val_dataset')

  val_data.create_dataset_folber()

in function   :  get_new_category_list
in function   :  get_category_name
in function   :  get_image_lable
loading annotations into memory...
Done (t=1.92s)
creating index...
index created!


100%|██████████| 5000/5000 [00:03<00:00, 1480.00it/s]


# برش داده تصاویر

In [ ]:
from pathlib import Path
from collections import OrderedDict
from typing import Optional, List, Tuple
import psutil
import cv2
import numpy as np
from pycocotools.coco import COCO



class CutImage:
    """
    Efficiently cut and composite objects from images based on COCO annotations,
    using a limited-size LRU cache with memory-aware eviction.
    """
    def __init__(
        self,
        image_dir: str,
        bake_dir: str,
        ann_file: str,
        category_id: int,
        destination_dir: str,
        cache_size: int = 1000,
        memory_threshold: float = 95.0 # [0, 100]
    ):
        # Initialize paths
        self.image_dir = Path(image_dir)
        self.bake_dir = Path(bake_dir)
        self.ann_file = Path(ann_file)
        self.destination_dir = Path(destination_dir)
        self.category_id = category_id
        self.bboxes_image :dict[str , List[List[float]]] = {}
        # LRU cache for loaded images
        self._cache: OrderedDict[Path, np.ndarray] = OrderedDict()
        self._cache_size = cache_size
        self._use_ram = True
        # Threshold for system memory usage (%) to trigger eviction
        self._memory_threshold = memory_threshold

        # Preload bake image paths for random selection
        self._bake_paths: List[Path] = (
            list(self.bake_dir.glob("*.jpg")) +
            list(self.bake_dir.glob("*.png")) +
            list(self.bake_dir.glob("*.jpeg"))
        )

        # Initialize COCO API
        self._coco = COCO(str(self.ann_file))

    def _read_image(self, path: Path) -> Optional[np.ndarray]:
        """
        Read image from disk with LRU caching. Evict oldest entries
        when cache is full or system memory usage exceeds threshold.
        """
        # Cache hit: move to end (most recently used)
        if path in self._cache:
            img = self._cache.pop(path)
            self._cache[path] = img
            return img

        # Check system memory and cache size for eviction
        mem_percent = psutil.virtual_memory().percent
        # Evict if memory usage too high
        while self._cache and mem_percent >= self._memory_threshold:
            evicted_path, _ = self._cache.popitem(last=False)
            if self._use_ram :
              self._use_ram  = False

              print(f"\n High memory usage {mem_percent:.1f}%  , evicted {evicted_path}")
            mem_percent = psutil.virtual_memory().percent
        # Evict oldest if cache full
        if len(self._cache) >= self._cache_size:

            evicted_path, _ = self._cache.popitem(last=False)
            # Optional: print(f"Evicted from cache due to size limit: {evicted_path}")

        # Cache miss: load from disk
        img = cv2.imread(str(path))
        if img is None:
            print(f"Warning: failed to read {path}")
            return None

        # Insert newly read image as most recently used
        self._cache[path] = img
        return img

    def _get_random_bake(self) -> Path:
        """Return a random bake image path."""
        if not self._bake_paths:
            raise FileNotFoundError(f"No bake images in {self.bake_dir}")
        return random.choice(self._bake_paths)

    @staticmethod
    def _merge_mask(
        shape: Tuple[int, int],
        segmentations: List[List[List[float]]]
        ) -> Tuple[np.ndarray, int, int, int, int]:
        """
        Combine multiple polygon segmentations into one mask and compute bounding box.
        Returns mask, x, y, w, h
        """
        mask = np.zeros(shape, dtype=np.uint8)
        points = []
        # Fill each polygon onto the mask
        for seg in segmentations:
            if isinstance(seg, list):
                pts = np.array(seg, dtype=np.int32).reshape(-1, 2)
                cv2.fillPoly(mask, [pts], 255)
                points.append(pts)
        if not points:
            # No valid segments, entire image
            return mask, 0, 0, shape[1], shape[0]
        all_pts = np.vstack(points)
        x, y, w, h = cv2.boundingRect(all_pts)
        return mask, x, y, w, h

    @staticmethod
    def _extract_and_crop(
        image: np.ndarray,
        mask: np.ndarray,
        x: int,
        y: int,
        w: int,
        h: int
        ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Apply mask to image and crop to bounding box.
        """
        extracted = cv2.bitwise_and(image, image, mask=mask)
        cropped_img = extracted[y:y+h, x:x+w]
        cropped_mask = mask[y:y+h, x:x+w]
        return cropped_img, cropped_mask

    @staticmethod
    def _add_padding(
        img: np.ndarray,
        mask: np.ndarray
        ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Add random padding around the cropped image and mask.
        """
        h, w = img.shape[:2]
        max_edge = max(h, w)
        pad_extra = random.randint(0, max_edge)
        target = max_edge + pad_extra

        pad_vert = target - h
        pad_horiz = target - w
        top = random.randint(0, pad_vert)
        left = random.randint(0, pad_horiz)
        bottom = pad_vert - top
        right = pad_horiz - left

        img_padded = cv2.copyMakeBorder(
            img, top, bottom, left, right,
            cv2.BORDER_CONSTANT, value=[0, 0, 0]
        )
        mask_padded = cv2.copyMakeBorder(
            mask, top, bottom, left, right,
            cv2.BORDER_CONSTANT, value=0
        )
        return img_padded, mask_padded

    @staticmethod
    def _resize(
        img: np.ndarray,
        size: int
        ) -> np.ndarray:
        """Resize image to (size, size)."""
        return cv2.resize(img, (size, size))

    def _save(
        self,
        img: np.ndarray,
        name: str,
        folber:str = None
        ) -> None:
        """Save image with JPEG quality 95."""
        if folber is not None:
          dir = self.destination_dir / folber
        else:
          dir = self.destination_dir
        dir = Path(dir)
        dir.mkdir(parents=True, exist_ok=True)
        out_path = dir / name
        cv2.imwrite(str(out_path), img, [cv2.IMWRITE_JPEG_QUALITY, 95])

    def create_cut_images(
        self,
        min_area: int = 5000,
        show: bool = False,
        save: bool = True
        ) -> None:
        """
        Process annotations, extract objects, composite with random bakes,
        and optionally display or save results.
        """
        # Get annotation IDs for the target category
        ann_ids = self._coco.getAnnIds(catIds=[self.category_id])
        anns = self._coco.loadAnns(ann_ids)

        for idx, ann in enumerate(tqdm(anns, desc="Processing annotations")):
            if ann.get('area', 0) < min_area:
                continue

            # Load front image
            img_info = self._coco.loadImgs(ann['image_id'])[0]
            front_path = self.image_dir / img_info['file_name']
            front_img = self._read_image(front_path)
            if front_img is None:
                continue

            # Merge all segmentations into one mask
            mask, x, y, w, h = self._merge_mask(
                front_img.shape[:2],
                ann.get('segmentation', [])  # list of polygons
            )

            # Extract and crop object
            #mask1 = cv2.bitwise_not(mask)
            cropped, mask_cropped = self._extract_and_crop(front_img, mask, x, y, w, h)
            padded, mask_padded = self._add_padding(cropped, mask_cropped)

            # Composite with random bake background
            mask_inv = cv2.bitwise_not(mask_padded)
            bake_img = self._read_image(self._get_random_bake())
            if bake_img is None:
                continue
            bake_resized = self._resize(bake_img, padded.shape[0])
            bake_part = cv2.bitwise_and(bake_resized, bake_resized, mask=mask_inv)
            final = cv2.bitwise_or(bake_part, padded)

            # Show or save
            if show:
                rgb = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(6, 6))
                plt.imshow(rgb)
                plt.axis('off')
                plt.show()


            if save:
                name = f"cut_indx_{idx}_imageName_{img_info['file_name']}"
                self._save(final, name, 'cut_image')


    def create_cut_box_images(
        self,
        min_area: int = 5000,
        max_area: int = 10000000,
        random_n:float = 0.2,
        show: bool = False,
        save: bool = True,
        random_size_box_cut = False
        ) -> None:

        """
        Process annotations, extract objects, composite with random bakes,
        and optionally display or save results.
        """
        # Get annotation IDs for the target category
        ann_ids = self._coco.getAnnIds(catIds=[self.category_id])
        anns = self._coco.loadAnns(ann_ids)
        self.bboxes_image :dict[str , List[List[float]]] = {}
        for idx, ann in enumerate(tqdm(anns, desc="Processing annotations")):

            # Load front image
            img_info = self._coco.loadImgs(ann['image_id'])[0]


            front_path = self.image_dir / img_info['file_name']
            front_img = self._read_image(front_path)
            if front_img is None:
                continue

            # Merge all segmentations into one mask

            bbox = ann.get('bbox', [])  # list of polygons
            x, y, w, h = [int(v) for v in bbox]



            if random_size_box_cut:
              random_size = random.randint(0, int(random_n * max(w,h)))
              size_cut  = max(w,h) + random_size

              #print('w')
              y1 = y - random.randint(0, max(int(w-h) , 0) + random_size)

              y = max(y1, 0)

              #print('h')
              x1 = x - random.randint(0, max(int(h-w), 0) + random_size)

              x = max(x1, 0)
              w = size_cut
              h = w

            # 4. برش مستقیم با استفاده از اسلایس NumPy
            y_end = min(y + h, front_img.shape[0])
            x_end = min(x + w, front_img.shape[1])
            cropped = front_img[y:y_end, x:x_end]

            # add bbox to list of box image
            self.bboxes_image.setdefault(img_info['file_name'], []).append(bbox)

            # Show or save
            if show:
                rgb = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(6, 6))
                plt.imshow(rgb)
                plt.axis('off')
                plt.show()

            if save and ann.get('area', 0) > min_area and ann.get('area', 0) < max_area :

                name = f"cut_area_{ann.get('area', 0)}_idx_{idx}_imageName_{img_info['file_name']}"
                self._save(cropped, name , 'box_image')

    def create_cut_box_bake(
        self,
        show: bool = False,
        save: bool = True,
        ) -> None:
      if len(self.bboxes_image) == 0 :
        print('Creat in  self.bboxes_image')
        print('you shoud de run function create_cut_box_images() ')
        return None

      for idx, img_name in enumerate(tqdm(self.bboxes_image.keys(), desc="Processing annotations")):

        bake_img = self._read_image(self.image_dir / img_name)
        if bake_img is None:
            continue




        h_img, w_img = bake_img.shape[:2]
        mask = np.zeros((h_img, w_img), dtype=np.uint8)

        for bbox in self.bboxes_image[img_name]:
            x, y, w, h = [int(v) for v in bbox]
            mask[y:y+h, x:x+w] = 255
            #bake_img[y:y+h, x:x+w] = 0
        random_img = self._read_image(self._get_random_bake())


        random_img = cv2.resize(random_img, (w_img, h_img))
        random_img = cv2.bitwise_and(random_img, random_img, mask=mask)
        mask_info = cv2.bitwise_not(mask)
        img_bake = cv2.bitwise_and(bake_img, bake_img, mask=mask_info)
        final = cv2.bitwise_or(random_img, img_bake, mask=None)


        # Show or save
        if show:
            rgb = cv2.cvtColor(final, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(6, 6))
            plt.imshow(rgb)
            plt.axis('off')
            plt.show()
        if save:
            name = f"bake_idx_{idx}_imageName_{img_name}"
            self._save(final, name, 'bake_image_Not')

    def cleanup(self):
        """
        Clear and delete all large internal attributes so that
        the garbage collector can reclaim their memory.
        """
        import gc

        # 1) LRU cache of loaded images
        if hasattr(self, '_cache'):
            self._cache.clear()    # حذف همه‌ی تصاویر از OrderedDict
            del self._cache        # حذف ارجاع به خودِ OrderedDict

        # 2) دیکشنری bboxes_image
        if hasattr(self, 'bboxes_image'):
            self.bboxes_image.clear()
            del self.bboxes_image

        # 3) لیست مسیرهای تصاویر bake
        if hasattr(self, '_bake_paths'):
            self._bake_paths.clear()
            del self._bake_paths

        # 4) شیء COCO (حاوی داده‌های آنتیشن)
        if hasattr(self, '_coco'):
            del self._coco

        # 5) حذف مقادیر مسیرها و پارامترهای دیگر (در صورت نیاز)
        for attr in ['image_dir', 'bake_dir', 'ann_file', 'destination_dir']:
            if hasattr(self, attr):
                delattr(self, attr)

        # 6) حذف تنظیمات کش/رم
        for attr in ['_cache_size', '_memory_threshold', '_use_ram', 'category_id']:
            if hasattr(self, attr):
                delattr(self, attr)

        # 7) حذف ماژول‌های import شده به عنوان attribute (در صورت بود)
        #    — معمولاً نیازی نیست مگر خودتان در __init__ ذخیره کرده باشید

        # 8) اجرای اجباری garbage collection
        gc.collect()


In [ ]:

if __name__ == "__main__":
  imm1 =  CutImage(image_dir = './data/val2017',
                bake_dir = '/content/data_set/dav_dataset/-1',
                ann_file = './data/annotations/instances_val2017.json',
                category_id = 1,
                destination_dir = '/content/cut4',
                cache_size= 10000,
                memory_threshold = 95
                )
  imm1.create_cut_box_images( min_area=10000)


loading annotations into memory...
Done (t=0.52s)
creating index...
index created!


Processing annotations:   0%|          | 12/11004 [00:00<01:34, 116.59it/s]


 High memory usage 30.9%  , evicted data/val2017/000000425226.jpg


Processing annotations: 100%|██████████| 11004/11004 [00:52<00:00, 211.50it/s]


In [ ]:

if __name__ == "__main__":
  imm1 =  CutImage(image_dir = './data/val2017',
                bake_dir = '/content/data_set/val_dataset/-1',
                ann_file = './data/annotations/instances_val2017.json',
                category_id = 1,
                destination_dir = '/content/cut4',
                cache_size= 10000,
                memory_threshold =95
                )
  imm1.create_cut_box_images( min_area=10000)
  imm1.create_cut_box_bake()


loading annotations into memory...
Done (t=0.70s)
creating index...
index created!


Processing annotations: 100%|██████████| 2693/2693 [00:24<00:00, 110.42it/s]


In [ ]:

if __name__ == "__main__":
  imm1 =  CutImage(image_dir = './data/val2017',
                bake_dir = '/content/data_set/val_dataset/-1',
                ann_file = './data/annotations/instances_val2017.json',
                category_id = 1,
                destination_dir = '/content/cut4',
                cache_size= 10000,
                memory_threshold = 95
                )

  imm1.create_cut_images( 10000 )

loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


Processing annotations:   0%|          | 23/11004 [00:00<00:49, 222.62it/s]


 High memory usage 32.4%  , evicted data/val2017/000000425226.jpg


Processing annotations: 100%|██████████| 11004/11004 [00:39<00:00, 280.53it/s]
